In [334]:
import copy, math
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split

# Linear Regression

## Key points:

### comparison of linear regression with/without ridge regularisation value
### comparison of optimal regularisation strength hyperparameter
### comparison of linear regression using basis function 

In [335]:
X , y = load_boston(return_X_y=True)
X = np.hstack([np.ones([X.shape[0], 1]), X])
test_size = 0.2
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)

In [336]:

# get optimal weights for linear regression
w = np.linalg.pinv(X_train) @ y_train

# get optimal weights for linear regression with regularisation
reg_strength = 0.001
w_reg = np.linalg.inv(X_train.T @ X_train + reg_strength * np.eye(X_train.shape[1])) @ X_train.T @ y_train

# Predict y using weights
y_pred = X_test @ w

# Predict y using weights with ridge regularisation
y_pred_reg = X_test @ w_reg

# check error on test set
mse_err = np.mean((y_test - y_pred)**2)
mse_err_reg = np.mean((y_test - y_pred_reg)**2)

In [337]:
mse_err

26.23360551980958

In [338]:
mse_err_reg

26.231687326279356

# Example below using Basis function on data that is exponential. 

## Regression is still linear on the weights on the basis function!
## Predictions can then be reverted back to original form by reverse basis function on data - here the basis function is to the the square root of the original data as is has exponential growth, so the reverse is to square the prediction of the linear regression to obtain the original data form.

In [339]:
# x1 = np.append(np.random.normal(2,1,5))
x1 = abs((np.random.normal(6,2,300)))
y1 = abs(x1**2+(np.random.normal(0,1,300)))
px.scatter(y=y1,x=x1)

In [340]:
def basis(data):

    ## Square Root
    basis = np.sqrt(data) 
    
    ## Squared
    # basis = data**2

    ## Radial basis
    # mean=np.mean(data)
    # std=np.std(data)
    # basis = -0.5*(np.exp(data-mean)/std)**2

    return basis

In [341]:
def reverse_basis(data):

    ## Square Root
    basis = data**2 
    
    ## Squared
    # basis = np.sqrt(data)

    ## Radial basis
    # mean=np.mean(data)  # Needs to be from original data!
    # std=np.std(data)        # Needs to be from original data!
    # basis = -0.5*(np.exp(data-mean)/std)**2

    return basis

In [342]:
X = np.c_[np.ones(len(x1)),x1]
test_size = 0.2
X_train, X_test, y_train, y_test = train_test_split(X, y1, test_size=test_size)

In [343]:

# get optimal weights for linear regression
w = np.linalg.pinv(X_train) @ y_train

# get optimal weights for linear regression with regularisation
reg_strength = 0.001
w_reg = np.linalg.inv(X_train.T @ X_train + reg_strength * np.eye(X_train.shape[1])) @ X_train.T @ y_train

# Predict y using weights
y_pred = X_test @ w

# Predict y using weights with ridge regularisation
y_pred_reg = X_test @ w_reg

# check error on test set
mse_err = np.mean((y_test - y_pred)**2)
mse_err_reg = np.mean((y_test - y_pred_reg)**2)

print("mse_err:",mse_err)
print("mse_err_reg:",mse_err_reg)

mse_err: 39.38958700968924
mse_err_reg: 39.386738724565134


In [345]:
# px.line(y=y_test)

fig = go.Figure()

fig.add_trace(go.Scatter(x=X_test[:,1],y=y_test,mode='markers'))
fig.add_trace(go.Scatter(x=X_test[:,1],y=y_pred_reg))
fig.show()

In [346]:
y1_basis = basis(y1)
X = np.c_[np.ones(len(x1)),x1]
test_size = 0.2
X_train, X_test, y_train, y_test = train_test_split(X, y1_basis, test_size=test_size)

In [347]:

# get optimal weights for linear regression
w = np.linalg.pinv(X_train) @ y_train

# get optimal weights for linear regression with regularisation
reg_strength = 0.001
w_reg = np.linalg.inv(X_train.T @ X_train + reg_strength * np.eye(X_train.shape[1])) @ X_train.T @ y_train

# Predict y using weights
y_pred = X_test @ w

# Predict y using weights with ridge regularisation
y_pred_reg = X_test @ w_reg

# check error on test set
mse_err = np.mean((y_test - y_pred)**2)
mse_err_reg = np.mean((y_test - y_pred_reg)**2)

print("mse_err:",mse_err)
print("mse_err_reg:",mse_err_reg)

mse_err: 0.016510035804848534
mse_err_reg: 0.016510090864710194


In [350]:
# px.line(y=y_test)

fig = go.Figure()

fig.add_trace(go.Scatter(x=X_test[:,1],y=reverse_basis(y_test),mode='markers'))
fig.add_trace(go.Scatter(x=np.sort(X_test[:,1]),y=np.sort(reverse_basis(y_pred_reg))))

fig.show()

## Below has functions relevant for gradient descent, to be added as an alternative instead of the moore-penrose matrix inversion of the matrix to obtaining optimal regression weights.

In [351]:
def cost(x1,y1,w,b):
    cost=0
    for i in range(len(x1)):
        cost += loss(x1[i],y1[i],w,b)
    cost/len(x1)
    return cost

In [352]:
def gradient(x,y,w,b):
    
    m = x.shape[0]
    n = 1
    dw = np.zeros(n)
    db = 0

    for i in range(m):
        pred = sigmoid(np.dot(x[i],w)+b)
        err = pred - y[i]
        # for j in range(n):
            # dw[j] += err*x[i,j]
        dw = dw + err*x[i]
        db = db + err
    
    dw = dw/m
    db = db/m

    return [db,dw]

In [353]:
def gradient_descent(x,y,w_in,b_in,alpha,iterations):

    j_hist = []
    w = copy.deepcopy(w_in)
    b = copy.deepcopy(b_in)
    
    for i in range(iterations):

        [db,dw] = gradient(x,y,w,b)

        w = w - alpha*dw
        b = b - alpha*db

        if i < 100000:
            j_hist.append(cost(x,y,w,b))

        
    return w,b,j_hist

In [354]:
w_tmp  = np.zeros(len(x1))
b_tmp  = 0.
alph = 0.1
iters = 10000

w_out, b_out, _ = gradient_descent(x1, y1, w_tmp, b_tmp, alph, iters) 
print(f"\nupdated parameters: w:{w_out}, b:{b_out}")

NameError: name 'sigmoid' is not defined

In [ ]:
pred_x=np.linspace(0,6,60).tolist()
pred_y=[]
sig_y=[]
for i in range(len(pred_x)):
    pred_y.append(w_out[0]*pred_x[i]+b_out[0])
    sig_y.append(sigmoid(pred_y[i]))